<a href="https://colab.research.google.com/github/shannu1063/shanmukhavarma_INFO5731_Fall2024/blob/main/Kothapalli_ShanmukhaVarma_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [1]:
import requests
import csv
import time
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def create_session():
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

def fetching_papers(query, limit=1000):
    base_url = "https://api.semanticscholar.org/graph/v1/paper/search"
    papers = []
    offset = 0
    batch_size = 100  # As API allows only a max of 100 papers per request
    session = create_session()

    with tqdm(total=limit, desc=f"Fetching papers for '{query}'") as pbar:
        while len(papers) < limit:
            params = {
                "query": query,
                "limit": min(batch_size, limit - len(papers)),
                "offset": offset,
                "fields": "title,abstract,year"
            }
            try:
                response = session.get(base_url, params=params)
                response.raise_for_status()

                data = response.json()
                batch_papers = data.get('data', [])

                if not batch_papers:
                    break

                papers.extend(batch_papers)
                offset += len(batch_papers)
                pbar.update(len(batch_papers))

                time.sleep(2)  # keeping 2 second delay between different requests so that server is not overwhelmed

            except requests.exceptions.RequestException as e:
                print(f"Error occurred: {e}")
                print("Waiting for 60 seconds before retrying...")
                time.sleep(60)

    return papers[:limit]

def saving_to_csv(papers, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['title', 'abstract', 'year'])
        writer.writeheader()
        for paper in papers:
            writer.writerow({
                'title': paper.get('title', ''),
                'abstract': paper.get('abstract', ''),
                'year': paper.get('year', '')
            })

def main():
    query = "machine learning"
    print(f"Processing the query: {query}")
    papers = fetching_papers(query)
    filename = f"{query.replace(' ', '_')}_abstracts.csv"
    saving_to_csv(papers, filename)
    print(f"Saved papers to {filename}")

if __name__ == "__main__":
    main()

Processing the query: machine learning


Fetching papers for 'machine learning':  10%|█         | 100/1000 [00:39<05:59,  2.51it/s]

Error occurred: HTTPSConnectionPool(host='api.semanticscholar.org', port=443): Max retries exceeded with url: /graph/v1/paper/search?query=machine+learning&limit=100&offset=100&fields=title%2Cabstract%2Cyear (Caused by ResponseError('too many 429 error responses'))
Waiting for 60 seconds before retrying...


Fetching papers for 'machine learning':  20%|██        | 200/1000 [02:23<10:20,  1.29it/s]

Error occurred: HTTPSConnectionPool(host='api.semanticscholar.org', port=443): Max retries exceeded with url: /graph/v1/paper/search?query=machine+learning&limit=100&offset=200&fields=title%2Cabstract%2Cyear (Caused by ResponseError('too many 429 error responses'))
Waiting for 60 seconds before retrying...


Fetching papers for 'machine learning':  30%|███       | 300/1000 [04:29<11:38,  1.00it/s]

Error occurred: HTTPSConnectionPool(host='api.semanticscholar.org', port=443): Max retries exceeded with url: /graph/v1/paper/search?query=machine+learning&limit=100&offset=300&fields=title%2Cabstract%2Cyear (Caused by ResponseError('too many 429 error responses'))
Waiting for 60 seconds before retrying...


Fetching papers for 'machine learning': 100%|██████████| 1000/1000 [07:38<00:00,  2.18it/s]

Saved papers to machine_learning_abstracts.csv


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [2]:
# Write code for each of the sub parts with proper comments.
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

# Downloading the required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def remove_noise(text_data):
    # Removing noise like special characters and punctuations
    text_data = re.sub(r'[^\w\s]', '', text_data)
    print("After removing all the noise:", text_data[:100])  # Printing the first 100 characters
    return text_data

def remove_numbers(text_data):
    # Removing numbers
    text_data = re.sub(r'\d+', '', text_data)
    print("After removing all the numbers:", text_data[:100])  # Printing the first 100 characters
    return text_data

def remove_stopwords(text_data):
    # Removing stopwords
    stop_words_set = set(stopwords.words('english'))
    words = nltk.word_tokenize(text_data)
    filtered_words_set = [word for word in words if word.lower() not in stop_words_set]
    text_data = ' '.join(filtered_words_set)
    print("After removing all the stopwords:", text_data[:100])  # Printing the first 100 characters
    return text_data

def lowercase_text(text_data):
    # text converted to all lower_case
    text_data = text_data.lower()
    print("After lowercasing:", text_data[:100])  # Printing the first 100 characters
    return text_data

def stem_text(text_data):
    # Stemming the words
    ps = PorterStemmer()
    words_set = nltk.word_tokenize(text_data)
    stemmed_words_set = [ps.stem(word) for word in words_set]
    text = ' '.join(stemmed_words_set)
    print("After stemming:", text_data[:100])  # Printing the first 100 characters
    return text_data

def lemmatize_text(text_data):
    # Lemmatization of the text
    lemmatizer = WordNetLemmatizer()
    words_set = nltk.word_tokenize(text_data)
    lemmatized_words_set = [lemmatizer.lemmatize(word) for word in words_set]
    text_data = ' '.join(lemmatized_words_set)
    print("After lemmatization:", text_data[:100])  # Printing the first 100 characters
    return text_data

def clean_text(text_data):
    if pd.isna(text_data):
        return ""
    text = remove_noise(text_data)
    text = remove_numbers(text_data)
    text = remove_stopwords(text_data)
    text = lowercase_text(text_data)
    text = stem_text(text_data)
    text = lemmatize_text(text_data)
    return text_data

def main():
    # Reading the CSV file
    input_file = "machine_learning_abstracts.csv"
    df = pd.read_csv(input_file)

    print("Original text data:")
    print(df['abstract'].iloc[0][:100])  # Printing the first 100 characters of the first abstract
    print("\n All Cleaning steps:")

    # Applying cleaning function to 'abstract' column
    df['cleaned_abstract'] = df['abstract'].apply(clean_text)

    # Saving the updated DataFrame to a new CSV file
    output_file = "cleaned_machine_learning_abstracts.csv"
    df.to_csv(output_file, index=False)
    print(f"\nCleaned data is saved to {output_file}")

    print("\nSome First few rows of the cleaned data:")
    print(df[['abstract', 'cleaned_abstract']].head())

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Original text data:
We present Fashion-MNIST, a new dataset comprising of 28x28 grayscale images of 70,000 fashion produ

 All Cleaning steps:
After removing all the noise: We present FashionMNIST a new dataset comprising of 28x28 grayscale images of 70000 fashion products
After removing all the numbers: We present Fashion-MNIST, a new dataset comprising of x grayscale images of , fashion products from 
After removing all the stopwords: present Fashion-MNIST , new dataset comprising 28x28 grayscale images 70,000 fashion products 10 cat
After lowercasing: we present fashion-mnist, a new dataset comprising of 28x28 grayscale images of 70,000 fashion produ
After stemming: We present Fashion-MNIST, a new dataset comprising of 28x28 grayscale images of 70,000 fashion produ
After lemmatization: We present Fashion-MNIST , a new dataset comprising of 28x28 grayscale image of 70,000 fashion produ
After removing all the noise: TensorFlow is a machine learning system that operates at large scale 

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [3]:
import pandas as pd
import spacy
from spacy import displacy
from collections import Counter

# Loading the spaCy model
nlp = spacy.load("en_core_web_sm")

def pos_tagging(document):
    pos_taggings = [(token.text, token.pos_) for token in document]
    pos_countings = Counter(tag for _, tag in pos_taggings)
    return pos_taggings, pos_countings

def simplified_constituency_parsing(document):
    def get_subtree(token):
        return f"({token.pos_} {token.text} {' '.join(get_subtree(child) for child in token.children)})"

    root_code = [token for token in document if token.head == token][0]
    return get_subtree(root_code)

def dependency_parsing(document):
    return [(token.text, token.dep_, token.head.text) for token in document]

def named_entity_recognition(document):
    entities_n = [(ent.text, ent.label_) for ent in document.ents]
    entity_countings = Counter(label for _, label in entities_n)
    return entities_n, entity_countings

def analyze_text(text):
    document = nlp(text)

    print("1. Parts of Speech (POS) Tagging:")
    pos_taggings, pos_countings = pos_tagging(document)
    print(f"POS Counts: {dict(pos_countings)}")
    print(f"Nouns: {pos_countings['NOUN'] + pos_countings['PROPN']}")
    print(f"Verbs: {pos_countings['VERB']}")
    print(f"Adjectives: {pos_countings['ADJ']}")
    print(f"Adverbs: {pos_countings['ADV']}")

    print("\n2. Simplified Constituency Parsing and Dependency Parsing:")
    example_sentence_tag = list(document.sents)[0]

    print("\nSimplified Constituency Parsing:")
    constituency_tree_tag = simplified_constituency_parsing(example_sentence_tag)
    print(constituency_tree_tag)

    print("\nDependency Parsing:")
    dependency_relations = dependency_parsing(example_sentence_tag)
    for word, dep, head in dependency_relations:
        print(f"{word} --{dep}--> {head}")

    print("\nExplanation:")
    print("This gives a very simple structure")
    print("of the sentence. Each pair of parentheses shows a constituent,")
    print("with the leftmost indicating the part of speech tag.")

    print("\nDependency Parsing: This gives the grammaar relationships of different words in the sentence.")
    print("Each word is connected to the governor (head) with a labeled arc showing the")
    print("relation.")

    print("\n3. Named Entity Recognition:")
    entities_n, entity_countings = named_entity_recognition(document)
    print(f"Entity Counts: {dict(entity_countings)}")
    print("\nEntities found:")
    for entity, label in entities_n:
        print(f"{entity}: {label}")

def main():
    # Reading the CSV file
    input_file_csv = "cleaned_machine_learning_abstracts.csv"
    df = pd.read_csv(input_file_csv)

    # Analysing the cleaned abstract
    some_sample_text = df['cleaned_abstract'].iloc[0]
    print("Analyzing the cleaned abstract:")
    print(some_sample_text)
    print("\nResults of Analysis:")
    analyze_text(some_sample_text)

if __name__ == "__main__":
    main()

Analyzing the cleaned abstract:
We present Fashion-MNIST, a new dataset comprising of 28x28 grayscale images of 70,000 fashion products from 10 categories, with 7,000 images per category. The training set has 60,000 images and the test set has 10,000 images. Fashion-MNIST is intended to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms, as it shares the same image size, data format and the structure of training and testing splits. The dataset is freely available at this https URL

Results of Analysis:
1. Parts of Speech (POS) Tagging:
POS Counts: {'PRON': 2, 'VERB': 8, 'PROPN': 4, 'PUNCT': 10, 'DET': 9, 'ADJ': 5, 'NOUN': 32, 'ADP': 11, 'NUM': 6, 'CCONJ': 3, 'AUX': 2, 'PART': 1, 'SCONJ': 1, 'ADV': 1}
Nouns: 36
Verbs: 8
Adjectives: 5
Adverbs: 1

2. Simplified Constituency Parsing and Dependency Parsing:

Simplified Constituency Parsing:
(VERB present (PRON We ) (PROPN MNIST (PROPN Fashion ) (PUNCT - ) (PUNCT , ) (NOUN compri

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

In [4]:
'''
cleaned_machine_learning_abstracts.csv

'''

'\ncleaned_machine_learning_abstracts.csv\n\n'

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [5]:
'''
The assignment teaches practical knowledge and gives hands-on experience with web scraping. Also scraping large data like 1000 or 10000 articles takes extremely long time to execute. Time provided is bit less as web scraping issues are creating blockage. More time could have been useful.
'''

'\nThe assignment teaches practical knowledge and gives hands-on experience with web scraping. Also scraping large data like 1000 or 10000 articles takes extremely long time to execute. Time provided is bit less as web scraping issues are creating blockage. More time could have been useful.\n'